In [1]:
!pip install nnfs

In [1]:
import numpy as np
import nnfs

nnfs.init()

## Dense Layers : fully connected layers

Weights defined as transpose for efficiency

## Activation functions:
            - ReLU
            - Softmax

## Loss function


In [2]:
class Layer_Dense:
    def __init__(self, num_inputs, num_neurons):
        # initialize weights and biases
        self.weights = 0.01 * np.random.randn(num_inputs, num_neurons) # <<- already transposed
        self.biases = np.zeros((1,num_neurons))

    def forward(self, inputs):
        # calculate output values from inputs,weights adn biases
        self.output = np.matmul(inputs, self.weights) + self.biases

In [3]:
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [4]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_vals = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        prob = exp_vals / np.sum(exp_vals, axis=1, keepdims=True)
        self.output = prob

In [5]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        mean_loss = np.mean(sample_losses)
        return mean_loss

class Lss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        num_samples = len(y_pred)
        # Clip to avoid log(0) 
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Depending on y shape we distinguish between y elements being class index or a one-hot array
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[ range(num_samples), y_true ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum( y_pred_clipped * y_true, axis=1 ) # because truth is one hot
        # 
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    

In [8]:
# Example run
from nnfs.datasets import spiral_data

np.random.seed(123)
X, y = spiral_data(samples=100, classes=3)
dense1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(3,3)
activation2 = Activation_Softmax()
loss_function = Lss_CategoricalCrossEntropy()

dense1.forward(X)
print(dense1.output[:10])

activation1.forward(dense1.output)
print(activation1.output[:10])

dense2.forward(activation1.output)
print(dense2.output[:10])

activation2.forward(dense2.output)
print(activation2.output[:10])

loss = loss_function.calculate(activation2.output, y)

print("Loss : ", loss)

predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)
print("Accuracy : ", accuracy)

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 9.4573094e-05 -5.9267721e-05  7.2208582e-05]
 [ 1.6830028e-04 -8.3797575e-05  2.0559385e-04]
 [ 6.5878841e-05  6.9999973e-05  4.4613457e-04]
 [ 2.8068494e-04 -9.6152013e-05  4.9797294e-04]
 [ 4.4868878e-04 -4.2396926e-04 -1.6528208e-04]
 [ 3.9569655e-05  2.1502166e-04  8.8323408e-04]
 [ 6.3656073e-04 -3.6693056e-04  5.9982738e-04]
 [ 6.7692413e-04 -6.7931623e-04 -3.9051610e-04]
 [ 8.1240491e-04 -4.6152930e-04  7.8957796e-04]]
[[0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [9.4573094e-05 0.0000000e+00 7.2208582e-05]
 [1.6830028e-04 0.0000000e+00 2.0559385e-04]
 [6.5878841e-05 6.9999973e-05 4.4613457e-04]
 [2.8068494e-04 0.0000000e+00 4.9797294e-04]
 [4.4868878e-04 0.0000000e+00 0.0000000e+00]
 [3.9569655e-05 2.1502166e-04 8.8323408e-04]
 [6.3656073e-04 0.0000000e+00 5.9982738e-04]
 [6.7692413e-04 0.0000000e+00 0.0000000e+00]
 [8.1240491e-04 0.0000000e+00 7.8957796e-04]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.39179065e-06

In [6]:
X.shape

(300, 2)

In [7]:
X[:5]

array([[-0.        ,  0.        ],
       [ 0.00570082,  0.00833853],
       [ 0.00680391,  0.01902179],
       [-0.01318397,  0.02728473],
       [ 0.00462581,  0.04013837]], dtype=float32)